<a href="https://colab.research.google.com/github/brandonjloeffler-lab/Semester-Project/blob/main/streamlit_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
from datetime import date

# --- CONFIGURATION ---
DATA_FILE_PATH = "data/bls_data.csv"
st.set_page_config(layout="wide", page_title="US Labor & Economic Dashboard (BLS Data)")

@st.cache_data
def load_data(path):
    """Loads the processed data from CSV, caches it for fast dashboard loading."""
    try:
        df = pd.read_csv(path, parse_dates=['Date'])
        # Ensure the date column is the index for easier slicing
        df.set_index('Date', inplace=True)
        return df
    except FileNotFoundError:
        st.error(f"Data file not found at {path}. Please run the data collection script first.")
        return pd.DataFrame()

# Load the data
df = load_data(DATA_FILE_PATH)

if not df.empty:

    # --- 1. SIDEBAR FILTERS (Interactivity) ---
    with st.sidebar:
        st.header("Dashboard Controls")

        min_date = df.index.min().to_pydatetime().date()
        max_date = df.index.max().to_pydatetime().date()

        # Date Range Slider
        st.subheader("Select Date Range")
        start_date, end_date = st.select_slider(
            'Filter data between:',
            options=pd.date_range(min_date, max_date, freq='MS').strftime('%Y-%m').tolist(),
            value=(pd.to_datetime(min_date).strftime('%Y-%m'),
                   pd.to_datetime(max_date).strftime('%Y-%m'))
        )

        # Convert slider strings back to datetime objects for filtering
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)

        # Filter the DataFrame
        df_filtered = df.loc[start_date:end_date]

        st.markdown("---")
        # Summary Section (Simple example)
        st.subheader("Latest Report Summary")
        latest_unemp = df.iloc[-1]['Unemployment_Rate_SA']
        prev_unemp = df.iloc[-2]['Unemployment_Rate_SA']

        st.metric(
            label=f"Unemployment Rate ({df.index[-1].strftime('%b %Y')})",
            value=f"{latest_unemp:.1f}%",
            delta=f"{latest_unemp - prev_unemp:.1f}"
        )


    # --- 2. MAIN DASHBOARD HEADER ---
    st.title("🇺🇸 US Labor & Economic Indicators Dashboard")
    st.markdown("""
        This interactive dashboard showcases key labor market, productivity, inflation, and trade
        trends using data collected from the US Bureau of Labor Statistics (BLS) Public API.
        Filter the date range using the sidebar to explore specific economic periods.
    """)
    st.markdown("---")


    # --- 3. EMPLOYMENT & LABOR PANEL ---
    st.header("💼 Labor Market Conditions")
    col1, col2 = st.columns(2)

    with col1:
        st.subheader("Unemployment Rate (SA)")
        fig_unemp = px.line(
            df_filtered,
            y='Unemployment_Rate_SA',
            title='Unemployment Rate Over Time',
            labels={'Unemployment_Rate_SA': 'Rate (%)'},
            height=400
        )
        st.plotly_chart(fig_unemp, use_container_width=True)

    with col2:
        st.subheader("Total Nonfarm Employment (SA)")
        fig_nonfarm = px.line(
            df_filtered,
            y='Total_Nonfarm_Employment_SA',
            title='Total Nonfarm Employment',
            labels={'Total_Nonfarm_Employment_SA': 'Employment (Thousands)'},
            height=400
        )
        st.plotly_chart(fig_nonfarm, use_container_width=True)

    st.markdown("---")


    # --- 4. PRODUCTIVITY PANEL ---
    st.header("🏭 Productivity and Hours")
    col3, col4 = st.columns(2)

    with col3:
        st.subheader("Output Per Hour - Non-farm Business")
        fig_output = px.line(
            df_filtered,
            y='Output_Per_Hour_NF',
            title='Output Per Hour Index',
            height=400
        )
        st.plotly_chart(fig_output, use_container_width=True)

    with col4:
        st.subheader("Total Private Average Weekly Hours")
        fig_hours = px.line(
            df_filtered,
            y='Avg_Weekly_Hours_Private_SA',
            title='Average Weekly Hours',
            height=400
        )
        st.plotly_chart(fig_hours, use_container_width=True)

    st.markdown("---")


    # --- 5. INFLATION & TRADE PANELS (Placeholders) ---
    st.header("💸 Inflation and Trade")

    st.subheader("CPI-U Less Food and Energy (Unadjusted)")
    # (Plotly chart for CPI goes here)
    st.markdown("*(Inflation Chart Placeholder)*")

    st.subheader("Imports vs. Exports (All Commodities)")
    # (Plotly chart for Imports/Exports goes here)
    st.markdown("*(Trade Balance Chart Placeholder)*")

    st.markdown("---")

    # --- 6. STATISTICAL ANALYSIS PANEL (OLS) ---
    st.header("🔬 Statistical Analysis: Unemployment vs. Employment")
    st.markdown(
        "**Objective:** Analyze the relationship between the **Unemployment Rate** (Dependent Variable) "
        "and **Total Nonfarm Employment** (Independent Variable) using Ordinary Least Squares (OLS)."
    )

    # Prepare data for OLS (only use non-null values for these two series)
    ols_df = df_filtered[['Unemployment_Rate_SA', 'Total_Nonfarm_Employment_SA']].dropna()

    if len(ols_df) > 5: # Ensure enough data points for meaningful regression
        try:
            # Define Variables
            Y = ols_df['Unemployment_Rate_SA']
            X = ols_df['Total_Nonfarm_Employment_SA']
            X = sm.add_constant(X) # Add the intercept term

            # Run OLS Regression
            model = sm.OLS(Y, X)
            results = model.fit()

            # Display key results
            col_ols1, col_ols2 = st.columns(2)
            with col_ols1:
                st.info(f"R-squared: **{results.rsquared:.4f}**")
                st.info(f"P-value (Nonfarm Employment): **{results.pvalues[1]:.4f}**")
                st.info(f"Coefficient (Employment): **{results.params[1]:.4e}**") # Scientific notation for small coef

            with col_ols2:
                # Create a scatter plot with the OLS line
                fig_ols = px.scatter(
                    ols_df,
                    x='Total_Nonfarm_Employment_SA',
                    y='Unemployment_Rate_SA',
                    title='OLS Regression: Unemployment vs. Nonfarm Employment',
                    trendline="ols", # Automatically adds the OLS line
                    height=500
                )
                st.plotly_chart(fig_ols, use_container_width=True)

        except Exception as e:
            st.error(f"Could not perform OLS regression with the selected data: {e}")
    else:
        st.warning("Not enough data points selected to perform OLS regression. Please widen the date range.")